# Inventário

Deve-se executar as seis primeiras células para carregar funções utilizadas para rodar o script

In [1]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from simple_history.utils import bulk_create_with_history
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from helpers.histories import bulk_update_with_history
import pytz

In [2]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [3]:
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from django.contrib.gis.db.models.functions import Distance
from turfpy.misc import nearest_point_on_line
from turfpy.measurement import length
from geojson import LineString, loads
from helpers.histories import bulk_update_with_history
from tqdm.notebook import tqdm

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [4]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [5]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [6]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [7]:
def findIndex(flist, func):
    for i,v in enumerate(flist):
        if func(v): 
            return i
    return -1

In [8]:
def point_to_km(lng, lat, road):
    point = Point(lng, lat, srid=4326)
#     roads = Road.objects.filter(company=company).annotate(distance=Distance(point, 'path')).order_by('distance')
#     road = roads[0]
    path = loads(road.path.geojson)
    point_geojson = loads(point.geojson)
    road_marks = list(road.marks.values())
    snap_point = nearest_point_on_line(path, point_geojson)
    

    road_markers = sorted([a for a in road_marks if 'index' in a or ('index' in a and a['index'] == 0)], key=lambda x: int(x['key']))
#     road_markers = sorted([a for a in road_marks if a['index'] or a['index'] == 0], key=lambda x: int(x['key']))
    rm_index = findIndex(road_markers, lambda x: x['index'] > snap_point['properties']['index'])
    
    if (rm_index == -1):
        rm_index = len(road_markers) - 1;
    
    km_marker_start = road_markers[rm_index - 1]
    km_marker_end = road_markers[rm_index]
    
    if km_marker_start['index'] == road_markers[-1]['index']: 
        snap = road_markers[-1]
        km = snap['km']
    elif km_marker_end['index'] == road_markers[0]['index']:
        snap = road_markers[0]
        km = snap['km']
    else:
        test = path['coordinates'][km_marker_start['index']:km_marker_end['index'] + 1]
        marker_path = LineString(test)
        snap = nearest_point_on_line(marker_path, point_geojson)        
        ls_length = length(marker_path, units="km")
        km = ((snap['properties']['location'] / ls_length)        
            * (km_marker_end['km'] - km_marker_start['km'])
            + km_marker_start['km'])
    
    return road, road.name, point, round(km, 3)

In [9]:
from collections import defaultdict
from django.db.models.signals import *


class DisableSignals(object):
    def __init__(self, disabled_signals=None):
        self.stashed_signals = defaultdict(list)
        self.disabled_signals = disabled_signals or [
            pre_init, post_init,
            pre_save, post_save,
            pre_delete, post_delete,
            pre_migrate, post_migrate,
        ]

    def __enter__(self):
        for signal in self.disabled_signals:
            self.disconnect(signal)

    def __exit__(self, exc_type, exc_val, exc_tb):
        for signal in list(self.stashed_signals):
            self.reconnect(signal)

    def disconnect(self, signal):
        self.stashed_signals[signal] = signal.receivers
        signal.receivers = []

    def reconnect(self, signal):
        signal.receivers = self.stashed_signals.get(signal, [])
        del self.stashed_signals[signal]

In [10]:
import re

def convert_string(string: str) -> str:
    if '_' in string:
        # Convert snake_case to camelCase
        return re.sub(r'_([a-z])', lambda x: x.group(1).upper(), string)
    else:
        # Convert camelCase to snake_case
        return re.sub(r'([A-Z])', lambda x: '_' + x.group(1).lower(), string)

 Verificando se está no ambiente de produção

In [11]:
Company.objects.all()

<QuerySet [<Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>, <Company: 385c3404-2c8b-4987-8015-9b93e044aa6d: CCR - AutoBAn>, <Company: 161ab539-eea0-4c7c-8e9f-99058261f850: CCR - SPVias>, <Company: be176150-b4da-4162-9eec-67f6db3f5612: CCR - RodoAnel>]>

In [12]:
company = Company.objects.get(name='CCR - Rio Sp')
user=User.objects.get(username='rlcs')
company,user

(<Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>,
 <User: rlcs - Kartado CS>)

In [13]:
reps=Reporting.objects.filter(occurrence_type__name__in=["Drenagem Superficial","Monitoração Drenagem Superficial"]).exclude(company=company)
reps.count()

11819

In [14]:
reps[0].form_data['latitudedecimal']

-22.669645

In [19]:
occs=OccurrenceType.objects.filter(company=company)
lista_latitude,lista_longitude=[],[]
for a in occs:
    for b in a.form_fields['fields']:
        if 'latitude' in b['apiName']:
            lista_latitude.append(convert_string(b['apiName']))
        if 'longitude' in b['apiName']:
            lista_longitude.append(convert_string(b['apiName']))


In [20]:
lista_latitude = list(set(lista_latitude))
lista_longitude = list(set(lista_longitude))

In [14]:
lista_latitude=['latidudeini']
lista_longitude=['longitudeini']

In [ ]:
rep_edit=[]
lat='latidudeini'
lon='longitudeini'
for a in tqdm(reps):
    try:
#         lat= next(valor for valor in lista_latitude if valor in list(a.form_data.keys()))
        if lat is not None and '-' in str(a.form_data[lat]):
            latitude = a.form_data[lat]
#         lon= next(valor for valor in lista_longitude if valor in list(a.form_data.keys()))
        if lon is not None and '-' in str(a.form_data[lon]):
            longitude = a.form_data[lon]
        road,_,point,km =point_to_km(float(longitude),float(latitude),a.road)
        a.point=point
        rep_edit.append(a)
    except Exception as e:
        print(e)      

  0%|          | 0/51491 [00:00<?, ?it/s]

In [ ]:
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=1000,user=user)

In [ ]:
rep_edit[0]

In [2]:
occ=OccurrenceType.objects.get(name='Monitoração Drenagem Superficial'
)

In [7]:
for a in occ.form_fields['fields']:
    if a['apiName'] == 'detalheelemento':
        for b in a['selectOptions']['options']:
            print(b)

{'name': 'DAD-02', 'value': '1'}
{'name': 'DAD-06', 'value': '2'}
{'name': 'DAD-10', 'value': '3'}
{'name': 'DAD-21L', 'value': '4'}
{'name': 'DAR-01', 'value': '5'}
{'name': 'DAR-03', 'value': '6'}
{'name': 'DAR-05L', 'value': '7'}
{'name': 'DEB', 'value': '8'}
{'name': 'DED', 'value': '9'}
{'name': 'DRT-01L', 'value': '10'}
{'name': 'DRZ-01L', 'value': '11'}
{'name': 'MFC-01', 'value': '12'}
{'name': 'MFC-02', 'value': '13'}
{'name': 'MFC-03', 'value': '14'}
{'name': 'MFC-04', 'value': '15'}
{'name': 'MFC-05', 'value': '16'}
{'name': 'MFC-06', 'value': '17'}
{'name': 'MFC-08', 'value': '18'}
{'name': 'SCC-01', 'value': '19'}
{'name': 'SCC-02', 'value': '20'}
{'name': 'SCC-03', 'value': '21'}
{'name': 'SCC-06L', 'value': '22'}
{'name': 'SCC-12L', 'value': '23'}
{'name': 'SMC-01L', 'value': '24'}
{'name': 'SRC-01L', 'value': '25'}
{'name': 'STC-01', 'value': '26'}
{'name': 'STC-02', 'value': '27'}
{'name': 'STC-03', 'value': '28'}
{'name': 'STC-04', 'value': '29'}
{'name': 'SZC-01', 'v

In [19]:
reps_history=reps.history.filter(history_user=user).order_by('-history_date')

AttributeError: 'QuerySet' object has no attribute 'history'

In [20]:
reps_history=Reporting.history.filter(occurrence_type__occurrence_kind=2,history_user=user,history_type='~',history_date).order_by('-history_date')

In [21]:
reps_history.count()

147999

In [24]:
reps_history[0].__dict__

{'_state': <django.db.models.base.ModelState at 0x7f551c554290>,
 'mobile_sync_id': None,
 'uuid': UUID('34ef1ff1-9d42-419c-9d8b-809c20824377'),
 'number': 'CCR-Inv-2023.76429',
 'road_name': 'BR-116 RJ',
 'km': 215.989,
 'end_km': 244.015,
 'km_reference': None,
 'project_km': 0.0,
 'project_end_km': 0.0,
 'point': <Point object at 0x7f551d4632e0>,
 'direction': '0',
 'lane': '29',
 'track': None,
 'branch': None,
 'address': {},
 'form_data': {'zone': '1',
  'index': None,
  'notes': '',
  'retro': [{'area': None,
    'type': 'Legenda: Pare',
    'color': '2',
    'width': None,
    'length': None,
    'description': 'PARE (P)'},
   {'area': None,
    'type': 'Legenda: Pare',
    'color': '2',
    'width': None,
    'length': None,
    'description': 'PARE (A)'},
   {'area': None,
    'type': 'Legenda: Pare',
    'color': '2',
    'width': None,
    'length': None,
    'description': 'PARE (R)'},
   {'area': None,
    'type': 'Legenda: Pare',
    'color': '2',
    'width': None,
    